In [1]:
import os
import numpy as np
import pandas as pd
import re

In [2]:
# set home directory
home_path = os.getcwd()[:os.getcwd().find('Code/python/obf_autax/bz')-1]
# set bz_path
bz_path = os.path.join(home_path, 'Data', 'obf_autman', 'sources', 'nutz_171_12_ps.xlsx')

print(home_path)
print(bz_path)

/home/philipp
/home/philipp/Data/obf_autman/sources/nutz_171_12_ps.xlsx


In [3]:
bz_data = pd.read_excel(bz_path)

In [4]:
bz_data.head()

,Waldort,WE-Typ,Umtriebszeit,Ertragssituation,Bewirtschaftungsform,Schutzwaldkategorie,Fläche in HA,Seehöhe,Exposition,Neigung,...,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39
0,201A0,WO,160,I,W,NaN,3.03,700,W,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201B1,WO,160,I,W,NaN,2.78,600,NW,70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201B2,WO,140,I,W,NaN,0.52,600,N,47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201B3,WO,140,I,W,NaN,1.98,600,NW,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201B4,WO,140,I,W,NaN,0.40,600,NW,70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# filter just WO
bz_data = bz_data.loc[bz_data['WE-Typ']=='WO', :]

In [6]:
bz_data.columns

Index(['Waldort', 'WE-Typ', 'Umtriebszeit', 'Ertragssituation',
       'Bewirtschaftungsform', 'Schutzwaldkategorie', 'Fläche in HA',
       'Seehöhe', 'Exposition', 'Neigung', 'Standorteinheit',
       'Alter der 1, Schicht', 'S', 'MA', 'FL', 'LH', 'NH', 'NH_ha', 'LH_ha',
       'SUM', 'DR', 'BH', 'ZP', 'SG', 'RU', 'Text', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39'],
      dtype='object')

In [7]:
bz_data = bz_data[['Waldort', 'MA', 'FL', 'LH', 'NH', 'DR', 'BH', 'ZP', 'SG', 'RU', 'Text']]

In [8]:
bz_data.iloc[3,:]

Waldort                                201B3
MA                                        DF
FL                                       1.8
LH                                        10
NH                                        70
DR                                         2
BH                                         1
ZP                                         3
SG                                         6
RU                                        36
Text       Mitte Dezennium, bei Bundesstraße
Name: 3, dtype: object

In [9]:
df = bz_data.copy()

In [10]:
# set raw_path
raw_path = os.path.join(home_path, 'Data', 'obf_autman', 'sources', '1310_171_12_20200821.XLS')
# read tsv
wo_data = pd.read_csv(raw_path, sep='\t', encoding = "ISO-8859-1", decimal=',', error_bad_lines=False)
# filter relevant data
wo_data = wo_data[wo_data['Best.-Schicht'] == 0]

### pernitz
wo_data = wo_data[wo_data['Abteilung'] > 200]

/home/philipp/miniconda3/envs/autax_rec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (36,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
# merge

wo_data.sort_values(by=['Abteilung', 'WE-Typ', 'Unterabteil.', 'Teilfl.'], ascending=[True,False,True,True], inplace=True)
wo_data = wo_data[['Waldort', 'WE-Typ']]

merge_data = pd.merge(wo_data, df, how='left', on='Waldort')

In [12]:
merge_data

,Waldort,WE-Typ,MA,FL,LH,NH,DR,BH,ZP,SG,RU,Text
0,201A0,WO,NaN,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201B1,WO,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201B2,WO,DF,0.5,10.0,70.0,2.0,1.0,3.0,6.0,36.0,"Mitte Dezennium, bei Bundesstraße"
3,201B3,WO,DF,1.8,10.0,70.0,2.0,1.0,3.0,6.0,36.0,"Mitte Dezennium, bei Bundesstraße"
4,201B4,WO,NaN,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1012,307O4,WO,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,307P1,WO,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1014,307P2,WO,NaN,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1015,30741,NG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
merge_data.fillna(0, inplace = True)
merge_data.iloc[:,4:11] = merge_data.iloc[:,4:11].astype(int)

In [15]:
merge_data

,Waldort,WE-Typ,MA,FL,LH,NH,DR,BH,ZP,SG,RU,Text
0,201A0,WO,0,2.7,0,0,0,0,0,0,0,0
1,201B1,WO,0,2.5,0,0,0,0,0,0,0,0
2,201B2,WO,DF,0.5,10,70,2,1,3,6,36,"Mitte Dezennium, bei Bundesstraße"
3,201B3,WO,DF,1.8,10,70,2,1,3,6,36,"Mitte Dezennium, bei Bundesstraße"
4,201B4,WO,0,0.4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1012,307O4,WO,0,0.3,0,0,0,0,0,0,0,0
1013,307P1,WO,0,3.5,0,0,0,0,0,0,0,0
1014,307P2,WO,0,1.1,0,0,0,0,0,0,0,0
1015,30741,NG,0,0.0,0,0,0,0,0,0,0,0


In [16]:
# set bz_path
save_path = os.path.join(home_path, 'Data', 'obf_autman', 'csv', 'nutz_autax_1310_171_12_20200821.XLS')
merge_data.to_csv(save_path, index=False)